In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff




In [2]:
yandex_world = pd.read_csv('yandex_world.csv')
yandex_world.rename(columns = {'Дата':'Date','Страна':'Country','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)
for i in range(yandex_world.shape[0]):
    a= yandex_world.Date[i].split('.')
    yandex_world.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))

yandex_world.tail(20)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
40746,Япония,2020-08-07,45764,30241,1042,1597,1383,8
40747,Япония,2020-08-08,47342,31248,1042,1578,1007,0
40748,Япония,2020-08-09,48782,32867,1047,1440,1619,5
40749,Япония,2020-08-10,49617,33450,1052,835,583,5
40750,Япония,2020-08-11,50302,34136,1058,685,686,6
40751,Япония,2020-08-12,51288,34969,1066,986,833,8
40752,Япония,2020-08-13,52471,36191,1073,1183,1222,7
40753,Япония,2020-08-14,53818,35498,1080,1347,-693,7
40754,Япония,2020-08-15,55051,39439,1093,1233,3941,13
40755,Япония,2020-08-16,56074,40560,1103,1023,1121,10


In [3]:
df3_temp = yandex_world.groupby('Date').sum()
df3_temp['Country']='Весь мир'

y5= pd.concat([yandex_world,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_world = y5
yandex_world.tail(10)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
208,Весь мир,2020-08-17,21881858,13888301,774034,209672,211433,4130
209,Весь мир,2020-08-18,22137926,14116451,780908,256068,228150,6874
210,Весь мир,2020-08-19,22411300,14333914,787672,273374,217463,6764
211,Весь мир,2020-08-20,22678483,14541573,793698,267183,207659,6026
212,Весь мир,2020-08-21,22949164,14712252,799249,270681,170679,5551
213,Весь мир,2020-08-22,23203490,14911319,804413,254326,199067,5164
214,Весь мир,2020-08-23,23420384,15137203,808674,216894,225884,4261
215,Весь мир,2020-08-24,23647189,15337252,813015,226805,200049,4341
216,Весь мир,2020-08-25,23889150,15570191,819414,241961,232939,6399
217,Весь мир,2020-08-26,24176836,15794441,825696,287686,224250,6282


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_world.tail(1).Date)[0]

print(yandex_world.iloc[-1,0])
print('Выбран день: ',myday)

Весь мир
Выбран день:  2020-08-26


In [5]:
X = pd.DataFrame()
country_dict = {}
yandex_world['Rt'] = 1
yandex_world['Rt2'] = 1
yandex_world['Rate_conf'] = 0
yandex_world['Rate_ill'] = 0

temp_df = yandex_world.groupby(['Country','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Country'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    country_dict[i]=j
    
print(country_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Австралия'), (1, 'Австрия'), (2, 'Азербайджан'), (3, 'Албания'), (4, 'Алжир'), (5, 'Ангола'), (6, 'Андорра'), (7, 'Антигуа и Барбуда'), (8, 'Аргентина'), (9, 'Армения'), (10, 'Афганистан'), (11, 'Багамские острова'), (12, 'Бангладеш'), (13, 'Барбадос'), (14, 'Бахрейн'), (15, 'Беларусь'), (16, 'Белиз'), (17, 'Бельгия'), (18, 'Бенин'), (19, 'Болгария'), (20, 'Боливия'), (21, 'Босния и Герцеговина'), (22, 'Ботсвана'), (23, 'Бразилия'), (24, 'Бруней'), (25, 'Буркина-Фасо'), (26, 'Бурунди'), (27, 'Бутан'), (28, 'Великобритания'), (29, 'Венгрия'), (30, 'Венесуэла'), (31, 'Весь мир'), (32, 'Восточный Тимор'), (33, 'Вьетнам'), (34, 'Габон'), (35, 'Гаити'), (36, 'Гайана'), (37, 'Гамбия'), (38, 'Гана'), (39, 'Гватемала'), (40, 'Гвинея'), (41, 'Гвинея-Бисау'), (42, 'Германия'), (43, 'Гондурас'), (44, 'Гренада'), (45, 'Греция'), (46, 'Грузия'), (47, 'ДР Конго'), (48, 'Дания'), (49, 'Джибути'), (50, 'Доминика'), (51, 'Доминиканская Республика'), (52, 'Египет'), (53, 'Замбия'), (54,

,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
40979,2020-08-22,61916,47418,1179,967,1154,4,1.121,1.071,0,0,Япония,13319,980.714286,1041.000000,13745.285714,14537.642857,-82,-191
40980,2020-08-23,62658,48319,1188,742,901,9,1.069,1.071,0,0,Япония,13151,940.571429,991.142857,13565.285714,14415.000000,-225,-168
40981,2020-08-24,63158,49089,1201,500,770,13,0.852,1.067,0,0,Япония,12868,920.142857,967.214286,13401.571429,14254.500000,-242,-283
40982,2020-08-25,63888,50092,1219,730,1003,18,0.694,1.061,0,0,Япония,12577,893.142857,970.428571,13264.571429,14073.714286,230,-291
40983,2020-08-26,64779,51407,1230,891,1315,11,0.669,1.054,0,0,Япония,12142,864.428571,963.642857,13028.571429,13851.500000,161,-435


In [6]:
# Добавляем колонки расчитанные сколько доля их в процентах от суммарного показателя в мире
mask = X.Country == 'Весь мир'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
world_date = list(temp_df.index)
world_conf = list(temp_df.Confirmed)
world_ill = list(temp_df.Remaining_ill)
world_dict_conf = dict(zip(world_date,world_conf))
world_dict_ill = dict(zip(world_date,world_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/world_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/world_dict_ill.get(X.loc[i,'Date'])


Confirmed  Remaining_ill
Date                                
2020-08-25   23889150        7499545
2020-08-26   24176836        7556699
40984
(40984, 19)
2020-01-22


In [7]:
# Проверка расчетов процентных показателей
mask = (X.Date>=datetime.date(2020,6,13))&(X.Country.isin(['Весь мир','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Country','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Country,Rate_conf,Rate_ill
28102,2020-08-07,875378,Россия,4.517372,2.576565
28103,2020-08-08,880563,Россия,4.484088,2.535220
28104,2020-08-09,885718,Россия,4.459431,2.546444
28105,2020-08-10,890799,Россия,4.434125,2.550231
28106,2020-08-11,895691,Россия,4.402687,2.546410
28107,2020-08-12,900745,Россия,4.368066,2.505044
28108,2020-08-13,905762,Россия,4.332313,2.449154
28109,2020-08-14,910778,Россия,4.293779,2.423043
28110,2020-08-15,915808,Россия,4.267572,2.377548
28111,2020-08-16,920719,Россия,4.248390,2.402935


In [8]:
X.to_csv('W.csv',index=False)